# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

## 2. Load the simulation data

In [2]:

# Set the working directory
indir = "/Users/an/Documents/0_mfpt/repptis1/"  

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_2108"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_repptismazegap2708"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/i_star/simulations/RETIS_flat_br_noswap"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_abl/paper-msm-short-term/simulations-and-analysis/trypsin-benzamidine/infrepptisanalysis/tistools-mfptanalysis/"
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS"

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file
# inputfile = indir + "/retis3.rst"

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/005', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/006', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/007', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/00

In [3]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

    
    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    #### CHANGE HERE ####
    # pe.set_orders(load=False, acc_only=True, save=True)        # for the 1st time you run this notebook for a certain simulation, this will store .npy files
    pe.set_orders(load=True, acc_only=True, save=False)                  # for the next times, you can read npy files (save=True/False is not important)
    # pe.set_orders(load=False, acc_only=True, save=False)     # if saving doesn't work

################################################################################
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/000
pathensemble info: 
{'cyclenumbers': array([    0,     1,     2, ..., 16166, 16167, 16168]),
 'flags': array(['ACC', 'REJ', 'REJ', ..., 'REJ', 'REJ', 'REJ'], dtype='<U3'),
 'generation': array(['sh', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[1.0, 1.0, 1.0], ['l_[0]', 'l_[0]', 'l_[0]']],
 'lambmaxs': array([1.03533, 1.03533, 1.03533, ..., 1.02224, 1.02224, 1.02224]),
 'lambmins': array([0.88933, 0.88933, 0.88933, ..., 0.94684, 0.94684, 0.94684]),
 'lengths': array([24,  1,  1, ...,  1,  1,  1]),
 'lmrs': array(['RMR', 'RMR', 'RMR', ..., 'RMR', 'RMR', 'RMR'], dtype='<U3'),
 'name': '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/000',
 'ncycle': 16169,
 'newpathnum

## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

In [4]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        data[i] = {}
        continue  #  [0-] is not used for Pcross calculations

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 2752
wLMR = 2111
wLML = 11138
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.15933277983244018
pLML = 0.8406672201675598
Local crossing probabilities:
p2R = 0.13192925442159864
p2L = 0.8680707455784014
doing pathensemble 2
Weights of the different paths:
wRMR = 10121
wRML = 953
wLMR = 881
wLML = 4214
Local crossing probabilities:
pRMR = 0.9139425681777136
pRML = 0.08605743182228644
pLMR = 0.17291462217860648
pLML = 0.8270853778213936
Local crossing probabilities:
p2R = 0.6804378749458841
p2L = 0.3195621250541159
doing pathensemble 3
Weights of the different paths:
wRMR = 6831
wRML = 1469
wLMR = 1647
wLML = 4737
Local crossing probabilities:
pRMR = 0.8230120481927711
pRML = 0.17698795180722893
pLMR = 0.25798872180451127
pLML = 0.7420112781954887
Local crossing probabilities:
p2R = 0.5773631163170798
p2L = 0.4226368836829202
doing pathensemble 4
Weights of the different paths:
wRMR = 1

### Generate pathlength distribution figures, as in PyRETIS reports.

In [5]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


### Compute Pcross using in-house functions

In [6]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [7]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar([i for i in range(len(Pcrossfull))], Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 0.0001490953708357295
which is the case!
[1.0, 0.15933277983244018, 0.027550967425393483, 0.02208471214378427, 0.013873410347435155, 0.005753189109422931, 0.0036440165563962398, 0.0031519768651947257, 0.002897240984707003, 0.0022149608555372094, 0.0006475760126250283, 0.00020295028461368047, 0.00018441088382212386, 0.0001755172466522791, 0.0001700349866595872, 0.00016457121907217058, 0.0001490953708357295]
[0.15933277983244018, 0.17291462217860648, 0.801594797118775, 0.6281906803725227, 0.41469177118995487, 0.633390713756974, 0.8649732558602538, 0.9191821858527552, 0.7645069454797898, 0.2923645404414911, 0.3133999417164894, 0.9086505307107763, 0.9517727100184431, 0.968765120822839, 0.9678668037987077, 0.9059626080204561]
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [8]:
# TODO DONT INCLUDE??

# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"]["LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])
print(c)
print([c[i]/c[i-1] for i in range(1,len(c))])

This should be the same as the repptis_report.pdf value: 0.0001490953708357295
[1.0, 0.15933277983244018, 0.027550967425393483, 0.02208471214378427, 0.013873410347435155, 0.005753189109422931, 0.0036440165563962398, 0.0031519768651947257, 0.002897240984707003, 0.0022149608555372094, 0.0006475760126250283, 0.00020295028461368047, 0.00018441088382212386, 0.0001755172466522791, 0.0001700349866595872, 0.00016457121907217058, 0.0001490953708357295]
[0.15933277983244018, 0.17291462217860648, 0.801594797118775, 0.6281906803725227, 0.41469177118995487, 0.633390713756974, 0.8649732558602538, 0.9191821858527552, 0.7645069454797898, 0.2923645404414911, 0.3133999417164894, 0.9086505307107763, 0.9517727100184431, 0.968765120822839, 0.9678668037987077, 0.9059626080204561]


## 4. Analysis using the MSM

### Construct transition matrix M

In [9]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 19.0]
N 17
NS 63


In [10]:
print("mm", pmms)
print("mp", pmps)
print("pm", ppms)
print("pp", ppps)
print("sum", np.array(pmms)+np.array(pmps))
print("sum", np.array(ppms)+np.array(ppps))
if N > 3:  
    M = construct_M(pmms, pmps, ppms, ppps, N)
elif N == 3:
    M = construct_M_N3(pmms, pmps, ppms, ppps, N)
else:
    raise ValueError("The amount of interfaces needs to be 3 at least!")

mm [0.8406672201675598, 0.8270853778213936, 0.7420112781954887, 0.9259626382005337, 0.9377109542804541, 0.7533737531781732, 0.7856794934242571, 0.7031975949713036, 0.9411988148598344, 0.941018950837682, 0.8299941072480849, 0.24060150375939848, 0.46252178965717605, 0.6972732331663885, 0.784712950600801, 0.8450271247739602]
mp [0.15933277983244018, 0.17291462217860648, 0.25798872180451127, 0.07403736179946627, 0.062289045719545875, 0.2466262468218267, 0.2143205065757428, 0.29680240502869637, 0.058801185140165614, 0.05898104916231804, 0.17000589275191513, 0.7593984962406015, 0.537478210342824, 0.30272676683361155, 0.21528704939919893, 0.15497287522603978]
pm [1.0, 0.08605743182228644, 0.17698795180722893, 0.2334936598163533, 0.30356027482823233, 0.08750693032711145, 0.21593406593406594, 0.1674447949526814, 0.6063205417607224, 0.5967365967365967, 0.3835762531105581, 0.15508885298869143, 0.13424821002386636, 0.20333333333333334, 0.4648707681106662, 0.5402489626556016]
pp [0.0, 0.91394256817

In [11]:
# We can print the transition matrix M and check that all rows sum to 1.
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)

M
shape (63, 63)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.84066722 0.15933278 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]]


### Look at this Markov model
*INCLUDE?*

In [12]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)
pprint(M)

[ 3.43117515e-16+3.64864280e-01j  3.43117515e-16-3.64864280e-01j
 -1.00000000e+00+0.00000000e+00j -9.93580157e-01+0.00000000e+00j
 -9.73146057e-01+0.00000000e+00j -9.83521809e-01+0.00000000e+00j
 -8.87669936e-01+0.00000000e+00j -8.39941008e-01+0.00000000e+00j
 -7.97772827e-01+0.00000000e+00j -7.13531897e-01+0.00000000e+00j
 -6.83366063e-01+0.00000000e+00j -6.57721976e-01+0.00000000e+00j
 -6.11715682e-01+0.00000000e+00j -5.67224215e-01+0.00000000e+00j
 -4.90634011e-01+0.00000000e+00j -4.64694440e-01+0.00000000e+00j
 -2.04666306e-01+0.00000000e+00j  2.04666306e-01+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  9.93580157e-01+0.00000000e+00j
  9.73146057e-01+0.00000000e+00j  9.83521809e-01+0.00000000e+00j
  8.87669936e-01+0.00000000e+00j  4.64694440e-01+0.00000000e+00j
  4.90634011e-01+0.00000000e+00j  8.39941008e-01+0.00000000e+00j
  7.97772827e-01+0.00000000e+00j  5.67224215e-01+0.00000000e+00j
  7.13531897e-01+0.00000000e+00j  6.83366063e-01+0.00000000e+00j
  6.57721976e-01+0.000000

In [13]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[0,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.84066722 0.         0.         0.         0.13178181 0.02755097
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
1.0
[0.         0.70672138 0.13394585 0.13178181 0.         0.
 0.         0.         0.02044313 0.00710784 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.

### Pcross with MSM

In [14]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

Z
state 0-     : 0
state B      : 1
state 0+- LML: 0.0
state 0+- LMR: 0.0009357482558988357
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.005411620163228794
state 1+- RML: 0.0
state 1+- RMR: 0.005411620163228794
state 2+- LML: 0.004945910029983622
state 2+- LMR: 0.00675106697633285
state 2+- RML: 0.004945910029983621
state 2+- RMR: 0.00675106697633285
state 3+- LML: 0.006431575945707908
state 3+- LMR: 0.010746843573561786
state 3+- RML: 0.006431575945707908
state 3+- RMR: 0.010746843573561786
state 4+- LML: 0.00973925594204715
state 4+- LMR: 0.02591525639084614
state 4+- RML: 0.00973925594204715
state 4+- RMR: 0.02591525639084614
state 5+- LML: 0.02100486524898711
state 5+- LMR: 0.04091511894313877
state 5+- RML: 0.02100486524898711
state 5+- RMR: 0.04091511894313877
state 6+- LML: 0.039172833760329526
state 6+- LMR: 0.0473021780337554
state 6+- RML: 0.039172833760329526
state 6+- RMR: 0.0473021780337554
state 7+- LML: 0.04554677567141674
state 7+- LMR: 0.05146115619056698
sta

### Pathlength analysis

In [15]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

0
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
1
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
2
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
3
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
4
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
5
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
6
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
7
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
8
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
9
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
10
Calculating path lengths.
Done.
Calculating first hitting

In [16]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

pyretis_data_17intf/
AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE

Ensemble 0 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/000):
--------------------------------------------------
  All accepted paths:    38.26 (n=1450, weighted)
------------------------------
  LML :     0.00 (n=   0, weighted)
  LMR :     0.00 (n=   0, weighted)
  RMR :    38.26 (n=1450, weighted)
  RML :     0.00 (n=   0, weighted)
  LM* :     0.00 (n=   0, weighted)
  *M* :     0.00 (n=   0, weighted)
  *** :     0.00 (n=   0, weighted)
  RM* :     0.00 (n=   0, weighted)
  L** :     0.00 (n=   0, weighted)
  **R :     0.00 (n=   0, weighted)
  R** :     0.00 (n=   0, weighted)

Ensemble 1 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/001):
--------------------------------------------------
  All accepted paths:    37.94 (n=1404, weighted)
------------------------------
  LML :    28.72 (n=1109, weighted)
  LMR :   

In [17]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

Collect tau
ensemble 0 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/000
ensemble 1 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/001
ensemble 2 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/002
ensemble 3 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/003
ensemble 4 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/004
ensemble 5 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/005
ensemble 6 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/006
ensemble 7 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/007
ensemble 8 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_17intf/008
ensembl

In [18]:
# Look at computed taus
print("tau")
print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
print("\ntau1")
print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
print("\ntaum")
print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
print("\ntau2")
print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)

tau
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan         36.3
1     001           26.7         52.1         60.8          nan
2     002          306.1        386.3        386.9        276.4
3     003          404.4        657.2        497.4        364.6
4     004          241.0        538.5        711.6        300.6
5     005          169.0        473.8        467.5        278.0
6     006          260.2        329.3        564.8        221.7
7     007          423.6        341.4        506.9        283.0
8     008          236.6        308.5        354.9        304.5
9     009          299.0        667.7        367.1        436.9
10    010          183.4        242.4        274.7        202.4
11    011          829.5       4364.9       3500.1       6832.7
12    012       280263.9      25350.1      46375.5       8320.1
13    013         4024.3       7235.5      105

In [19]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
tau_m = tau-tau1-tau2  # yes, this is the same thing as taum

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

tau
[3.62557363e+01 2.67205962e+01 5.21440076e+01 6.08063227e+01
 3.06129331e+02 3.86286039e+02 3.86917104e+02 2.76413299e+02
 4.04417775e+02 6.57238616e+02 4.97351940e+02 3.64612502e+02
 2.40998024e+02 5.38455201e+02 7.11649813e+02 3.00604107e+02
 1.68983312e+02 4.73843596e+02 4.67477366e+02 2.77998655e+02
 2.60197819e+02 3.29306106e+02 5.64771911e+02 2.21651241e+02
 4.23630657e+02 3.41368750e+02 5.06899491e+02 2.83030133e+02
 2.36595414e+02 3.08452578e+02 3.54896006e+02 3.04536829e+02
 2.99013318e+02 6.67704134e+02 3.67077439e+02 4.36910550e+02
 1.83367238e+02 2.42377183e+02 2.74738281e+02 2.02358382e+02
 8.29491303e+02 4.36494627e+03 3.50010009e+03 6.83269723e+03
 2.80263906e+05 2.53501337e+04 4.63755052e+04 8.32013958e+03
 4.02433166e+03 7.23546162e+03 1.05346511e+04 5.76853136e+03
 8.40156504e+03 7.02088603e+03 4.97356557e+03 5.71486151e+03
 4.86308124e+03 5.16829302e+03 4.98421535e+03 4.03023946e+03
 3.43683865e+03 3.95760093e+03 0.00000000e+00]


tau1
[   0.            0.       

## 5. Flux calculation

### Collect tau for [0+]

In [20]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

G
state 0-     : 0.0
state B      : 0.0
state 0+- LML: 26.720596157299337
state 0+- LMR: 5304.755151411118
state 0+- RML: 0.0
state 1+- LML: 244.67821547223542
state 1+- LMR: 29206.560474259797
state 1+- RML: 332.9202518363064
state 1+- RMR: 29091.862196070608
state 2+- LML: 26966.72153997814
state 2+- LMR: 34342.30507157795
state 2+- RML: 27072.464315052566
state 2+- RMR: 34039.032248310825
state 3+- LML: 32988.36962705697
state 3+- LMR: 43095.86001810478
state 3+- RML: 33464.31081223558
state 3+- RMR: 42848.02107612332
state 4+- LML: 40775.83509109849
state 4+- LMR: 70292.8917805975
state 4+- RML: 41074.88031448687
state 4+- RMR: 70116.35610165606
state 5+- LML: 61509.36118822945
state 5+- LMR: 95488.45051940343
state 5+- RML: 61815.858807371056
state 5+- RMR: 95385.34597693663
state 6+- LML: 92813.53723183081
state 6+- LMR: 103993.51295998038
state 6+- RML: 92908.06437255605
state 6+- RMR: 103928.95134231457
state 7+- LML: 101730.17282148261
state 7+- LMR: 108347.17201551377
state 7

### The flux

In [21]:
flux = 1/(tau[0]+h1[0][0])
dt = 0.002 # Change if needed
sc = 5
flux /= (dt*sc)
# print(flux/(dt*sc), "1/time")
print(flux, "1/ps")

0.11062678089591603 1/ps


## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [22]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0], "1/ps")

The rate constant k is:  1.6493940922038444e-05 1/ps


## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [23]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

Eigenvalues of Mp:
[-4.77893379e-16+3.64864280e-01j -4.77893379e-16-3.64864280e-01j
  9.99969231e-01+0.00000000e+00j  9.93663159e-01+0.00000000e+00j
  9.83427893e-01+0.00000000e+00j  9.73189833e-01+0.00000000e+00j
  8.87663253e-01+0.00000000e+00j  8.39951537e-01+0.00000000e+00j
  7.97764083e-01+0.00000000e+00j  7.13544182e-01+0.00000000e+00j
  6.83347948e-01+0.00000000e+00j  6.57732037e-01+0.00000000e+00j
  6.11714042e-01+0.00000000e+00j  5.67224437e-01+0.00000000e+00j
  4.90634002e-01+0.00000000e+00j  4.64694441e-01+0.00000000e+00j
 -9.99969231e-01+0.00000000e+00j -9.93663159e-01+0.00000000e+00j
 -9.73189833e-01+0.00000000e+00j -9.83427893e-01+0.00000000e+00j
 -8.87663253e-01+0.00000000e+00j -8.39951537e-01+0.00000000e+00j
 -7.97764083e-01+0.00000000e+00j -7.13544182e-01+0.00000000e+00j
 -6.83347948e-01+0.00000000e+00j -6.57732037e-01+0.00000000e+00j
 -6.11714042e-01+0.00000000e+00j -5.67224437e-01+0.00000000e+00j
 -4.90634002e-01+0.00000000e+00j -4.64694441e-01+0.00000000e+00j
  2.04

In [24]:
k_flux_pcross = flux * y1[0][0]
k_mfpt = 1 / (mfpt*dt*sc)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")


Rate constant from P_cross × flux: 1.6493940922e-05 [1/ps]
Rate constant from MFPT:           1.6493940922e-05 [1/ps]
Relative difference:               0.00%
